In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

from VQLSSVM import VQLSSVM

np.set_printoptions(precision=4, suppress=True)

In [2]:
shots: int = 10000
gamma: float = 0.01 # regularization parameter
subsetSize: int = 7
classToFilterOut: int = 2

In [3]:
def prepareDataset(normalizeValues: bool = False, dataset: str = "iris") -> (np.ndarray, np.ndarray, np.array, np.array):
    if dataset == "iris":
        X,y = datasets.load_iris(return_X_y=True)
        X = X[y!=classToFilterOut]
        y = y[y!=classToFilterOut]
    elif dataset == "breastCancer":
        X,y = datasets.load_breast_cancer(return_X_y=True)
    elif dataset == "dummyData":
        X, y = datasets.make_classification(
            n_samples=50, n_features=2, n_informative=2, n_redundant=0, random_state=45
        )
    else: 
        raise ValueError("Dataset not yet implemented")
    
    y: np.array = np.where(y == 0, -1, 1) # might cause some problems in a future

    if normalizeValues:
        max = np.max(X, axis=0)
        min = np.min(X, axis=0)
        X = (2*X - min - max) / (max - min)
    return train_test_split(X, y, test_size=(X.shape[0]-subsetSize)/(X.shape[0]))

In [4]:
xTrain, xTest, yTrain, yTest = prepareDataset(normalizeValues=True, dataset="iris")
print(xTrain)

[[-0.8519 -0.75   -0.8537 -0.7647]
 [-0.1852  0.5833 -0.6585 -0.6471]
 [-0.6296 -0.1667 -0.8049 -1.    ]
 [ 0.8519 -0.3333  0.8537  0.5294]
 [-1.     -0.1667 -0.9512 -1.    ]
 [-0.037  -0.1667  0.5122  0.4118]
 [ 1.      0.      0.8049  0.5294]]


In [5]:
vqlssvmVectors: VQLSSVM = VQLSSVM(gamma, shots)

methods = ['COBYLA','SLSQP', 'BFGS', 'L-BFGS-B', 'trust-constr']
costs = []
for method in methods:
    print("Method: " + method)
    vqlssvmVectors.train(xTrain, yTrain, quantumSimulation=False, verbose=False, iterations = 200, method=method)

    vqlssvmVectors.plotCost()
    cost = vqlssvmVectors.getCostHistory()
    print("Cost history:")
    print(cost)
    costs.append(cost)

Method: SLSQP
Time to prepare circuits: 6.287042617797852


KeyboardInterrupt: 

In [ ]:
import csv
# where one column is iteration number and the other is cost

with open('costOptimizers3.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['Iteration'] + methods)
    for i in range(200):
        values = [i]
        for j in range(len(costs)):
            if i >= len(costs[j]):
                values.append(costs[j][-1])
            else:
                values.append(costs[j][i])
        writer.writerow(values)

In [ ]:
# import pandas as pd

# # Load the first CSV file
# df1 = pd.read_csv('costOptimizers1.csv')

# # Load the second CSV file and drop its first column
# df2 = pd.read_csv('costOptimizers2.csv')
# df2 = df2.iloc[:, 1:]  # Remove the first column

# # Combine the two DataFrames
# combined_df = pd.concat([df1, df2], ignore_index=False, axis=1)

# # Save the combined DataFrame to a new CSV file
# combined_df.to_csv('costOptimizers.csv', index=False)